In [4]:
import os

In [5]:
os.chdir('../')

In [22]:
%pwd

'c:\\Users\\Naitik\\OneDrive\\Desktop\\Masters\\MLops\\car-pred-MLoperations'

In [7]:
os.chdir('car-pred-MLoperations')

In [8]:
%pwd

'c:\\Users\\Naitik\\OneDrive\\Desktop\\Masters\\MLops\\car-pred-MLoperations'

In [9]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path

In [10]:
from carPred.constants import *
from carPred.utils.common import read_yaml, create_directories

In [11]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
        )

        return data_transformation_config

In [13]:
import os
from carPred import logger
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [24]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    
    ## Note: You can add different data transformation techniques such as Scaler, PCA and all
    #You can perform all kinds of EDA in ML cycle here before passing this data to the model

    # I am only adding train_test_spliting cz this data is already cleaned up


    def eda(self):
        data = pd.read_csv(self.config.data_path)
        data = data.drop(['ind'], axis = 1)

        cat = []
        num = []
        for i in data.columns:
            if data[i].dtypes == 'O':
                cat.append(i)
            else:
                num.append(i)
        data_n=data[num]   # new dataframe just type numeric
        data_c=data[cat]

        label_encoders = {}
        categorical_columns = data_c.columns  # I would recommend using columns names here if you're using pandas. If you're using numpy then stick with range(n) instead

        for column in categorical_columns:
            label_encoders[column] = LabelEncoder()
            data_c[column] = label_encoders[column].fit_transform(data_c[column])

        frames = [data_c, data_n]
  
        data = pd.concat(frames,axis=1)
        print(data.head())
        return data



    def train_test_spliting(self):
        data = DataTransformation.eda(self)

        # Split the data into training and test sets. (0.75, 0.25) split.
        
        

        
        train, test = train_test_split(data)

        train.to_csv(os.path.join(self.config.root_dir, "train.csv"),index = False)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"),index = False)

        logger.info("Splited data into training and test sets")
        logger.info(train.shape)
        logger.info(test.shape)

        print(train.shape)
        print(test.shape)

In [25]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.train_test_spliting()
except Exception as e:
    raise e

[2024-02-22 19:45:55,788: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-02-22 19:45:55,791: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-22 19:45:55,794: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-02-22 19:45:55,796: INFO: common: created directory at: artifacts]
[2024-02-22 19:45:55,797: INFO: common: created directory at: artifacts/data_transformation]
   Make  Model  Condition  Year  Mileage     Price
0     1      4          0  2022    18107  19094.75
1     4      4          0  2014    13578  27321.10
2     0      2          2  2016    46054  23697.30
3     1      2          0  2022    34981  18251.05
4     0      2          0  2019    63565  19821.85
[2024-02-22 19:45:55,817: INFO: 3534830584: Splited data into training and test sets]
[2024-02-22 19:45:55,819: INFO: 3534830584: (750, 6)]
[2024-02-22 19:45:55,820: INFO: 3534830584: (250, 6)]
(750, 6)
(250, 6)


C:\Users\Naitik\AppData\Local\Temp\ipykernel_10108\3534830584.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_c[column] = label_encoders[column].fit_transform(data_c[column])
C:\Users\Naitik\AppData\Local\Temp\ipykernel_10108\3534830584.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_c[column] = label_encoders[column].fit_transform(data_c[column])
C:\Users\Naitik\AppData\Local\Temp\ipykernel_10108\3534830584.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice